<a href="https://colab.research.google.com/github/JeromeMberia/user_review_analysis/blob/main/user_review_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing the libraries
import pandas as pd
import numpy as np
import string

In [ ]:
#uploading and readin the dataset
data = pd.read_csv("/content/user_review_data.csv")
data.head()

,review_body,star_rating
0,Good luck finding a reasonably priced band rep...,1
1,No i get dark on the first week with me!! I wi...,1
2,I dont know if mine was a mistake but it clear...,1
3,The rod that holds the bracelet broke several ...,1
4,"I bought 2 watches , one watch doesnot work at...",1


In [ ]:
#checking the top 5 rows
data.head()

,review_body,star_rating
0,Good luck finding a reasonably priced band rep...,1
1,No i get dark on the first week with me!! I wi...,1
2,I dont know if mine was a mistake but it clear...,1
3,The rod that holds the bracelet broke several ...,1
4,"I bought 2 watches , one watch doesnot work at...",1


In [ ]:
#checking the bottom 5 rows
data.tail()

,review_body,star_rating
14995,"I have a TAG Heuer carrera twin time watch, wh...",5
14996,It is hard sometimes to judge a product by its...,5
14997,reloj en perfecto estado llego todo a tiempo l...,5
14998,This is my third ironman. There are more expe...,5
14999,i use it everyday,5


In [ ]:
#Getting the dataset shape.
data.shape

(15000, 2)

In [ ]:
##Get the summary of this dataframe
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review_body  15000 non-null  object
 1   star_rating  15000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 234.5+ KB


observation: our dataset comprise of object and integer datatype. both colunms has 15000 values, this is a clue of no missing values. 

In [ ]:
#checking the body_review variable values
data['review_body'].sample(n=10, random_state=610)

3293     The delivery was prompt. I got the correct pro...
5328                                    not silent running
13737                                         good quality
3751     not happy<br /><br />  the band is very stiff ...
14280                                            Excellent
7552                 A backlight would be a useful feature
13104    So far so good with this product. My fiancée i...
11599    This is a nice watch, however it is a little h...
6004     IT IS OK. SEEMS TO BE A GOOD QUALITY. I RECOME...
2401     Similar to this watch. Undelivered to this dat...
Name: review_body, dtype: object

observation: the values comprises of words, symbols, panctuations and so on.

In [ ]:
#Using groupby will check the data
data.groupby('star_rating').describe()

review_body                  
                  count unique   top freq
star_rating                              
1                  3000   2935   bad    3
2                  3000   2896    ok    4
3                  3000   2905  good    7
4                  3000   2929  Good   11
5                  3000   2896  good    8

In [ ]:
#previewing the dataset variables
data.columns

Index(['review_body', 'star_rating'], dtype='object')

observation: the two colunms are, review_body, and star_rating.

## Pre processing the data

Here we will check for missing values, duplicates, outliers and remove any unnecessary variables/features/columns if any. other preprocessing procedures are tokenizing(spliting),stemming and lemmatization.

In [ ]:
#checking for the missing values
data_corpus = data.copy()
data_corpus.head(10)

,review_body,star_rating
0,Good luck finding a reasonably priced band rep...,1
1,No i get dark on the first week with me!! I wi...,1
2,I dont know if mine was a mistake but it clear...,1
3,The rod that holds the bracelet broke several ...,1
4,"I bought 2 watches , one watch doesnot work at...",1
5,"This watch would have been fantastic, if it ha...",1
6,I have this watch. It looks and feels heavy du...,1
7,What the hell! I just got the watch today but ...,1
8,I am mechanically inclined but cannot get this...,1
9,It didnt work right out from the box. I had to...,1


observation: there is no missing values

In [ ]:
#Checking for duplicates
data_corpus.duplicated().sum()

439

observation: there are 439 duplicates.

In [ ]:
#dropping the duplicates
data_corpus.drop_duplicates(inplace=True)
#
#confirming no duplicates
data_corpus.duplicated().any()

False

observation: duplicates have been dropped.

observation, there are several panctuations. 

In [ ]:
import re
import string

def clean_text_round1(text):
  '''Remove HTML tags, remove punctuation, make text lowercase, remove punctuation and remove words containing numbers.'''
  text = re.sub('<.*?>','', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  text = text.lower()
  text = re.sub('\d', '', text)
  return text

fix = lambda x: clean_text_round1(x)
data_corpus['review_body'] = pd.DataFrame(data_corpus.review_body.apply(fix))
data_corpus

,review_body,star_rating
0,good luck finding a reasonably priced band rep...,1
1,no i get dark on the first week with me i will...,1
2,i dont know if mine was a mistake but it clear...,1
3,the rod that holds the bracelet broke several ...,1
4,i bought watches one watch doesnot work at a...,1
...,...,...
14995,i have a tag heuer carrera twin time watch who...,5
14996,it is hard sometimes to judge a product by its...,5
14997,reloj en perfecto estado llego todo a tiempo l...,5
14998,this is my third ironman there are more expen...,5


In [ ]:
data_corpus2 = data_corpus.copy()

In [ ]:
from nltk.corpus import stopwords

In [ ]:
pip install nltk


In [ ]:
pip install wordnet

In [ ]:
nltk.download()

NameError: ignored

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
data_corpus['review_body'] = data_corpus['review_body'].apply(nltk.word_tokenize)

In [ ]:
data_corpus['review_body']

0        [good, luck, finding, a, reasonably, priced, b...
1        [no, i, get, dark, on, the, first, week, with,...
2        [i, dont, know, if, mine, was, a, mistake, but...
3        [the, rod, that, holds, the, bracelet, broke, ...
4        [i, bought, watches, one, watch, doesnot, work...
                               ...                        
14995    [i, have, a, tag, heuer, carrera, twin, time, ...
14996    [it, is, hard, sometimes, to, judge, a, produc...
14997    [reloj, en, perfecto, estado, llego, todo, a, ...
14998    [this, is, my, third, ironman, there, are, mor...
14999                               [i, use, it, everyday]
Name: review_body, Length: 14561, dtype: object

In [ ]:
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
 
data_corpus['review_body'] = data_corpus['review_body'].apply(lambda x: [stemmer.stem(y) for y in x])

In [ ]:
data_corpus['review_body']


0        [good, luck, find, a, reason, price, band, rep...
1        [no, i, get, dark, on, the, first, week, with,...
2        [i, dont, know, if, mine, wa, a, mistak, but, ...
3        [the, rod, that, hold, the, bracelet, broke, s...
4        [i, bought, watch, one, watch, doesnot, work, ...
                               ...                        
14995    [i, have, a, tag, heuer, carrera, twin, time, ...
14996    [it, is, hard, sometim, to, judg, a, product, ...
14997    [reloj, en, perfecto, estado, llego, todo, a, ...
14998    [thi, is, my, third, ironman, there, are, more...
14999                               [i, use, it, everyday]
Name: review_body, Length: 14561, dtype: object

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

data_corpus['review_body'] = data_corpus['review_body'].apply(lambda x: ' '.join(x))

cv = CountVectorizer(stop_words='english')
text_1 = cv.fit_transform(data_corpus['review_body'])

In [ ]:
data_DTM = pd.DataFrame(text_1.toarray(), columns=cv.get_feature_names(), index=data_corpus.index)
data_DTM['star_rating'] = data_corpus2['star_rating']
data_DTM.head()


,aa,aaa,aaaaa,aaaaaand,aaah,aaath,aah,aalk,ab,aback,abandon,abb,abbrevi,abbreviationsth,abc,abcd,abco,abd,abeast,abel,abil,abilityon,abit,abl,ablack,abnorm,aboitna,abomin,abort,abound,abous,aboutbut,abouti,aboutit,aboutth,abov,aboveeven,abovement,abras,abroad,...,zebra,zen,zenith,zeno,zenwatch,zeon,zeppelin,zero,zerodarkthirti,zeroout,zeror,zerostar,zest,zesu,zeu,zig,zillion,ziploc,ziplock,zipper,zippo,zissou,zodiac,zombi,zone,zonegot,zonesbut,zoneth,zonevisit,zoo,zoom,zulu,zumiez,zurich,ágil,ágora,él,ít,ñjjñlk,star_rating
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
data_DTM.star_rating.unique()

array([1, 2, 3, 4, 5])

In [ ]:
data_DTM.shape

(14561, 17284)